In [1]:
from ipyevents import Event 
##########
#########
import statistics
from ipywidgets import *
import io
import cv2
import PIL.Image
from scipy.ndimage import filters
import numpy as np
##########
from IPython.display import display, clear_output
from ipywebrtc import CameraStream, ImageRecorder

In [2]:
#############################################
def adjust_brightness(img, brightness_factor):
    """Adjust brightness of an Image.
    Args:
        img (numpy ndarray): numpy ndarray to be adjusted.
        brightness_factor (float):  How much to adjust the brightness. Can be
            any non negative number. 0 gives a black image, 1 gives the
            original image while 2 increases the brightness by a factor of 2.
    Returns:
        numpy ndarray: Brightness adjusted image.
    """
    table = np.array([ i*brightness_factor for i in range (0,256)]).clip(0,255).astype('uint8')
    if img.shape[2]==1:
        return cv2.LUT(img, table)[:,:,np.newaxis]
    else:
        return cv2.LUT(img, table) 
##################################################
####################################################
def cible_petite(img,xc,yc):
    if len(img.shape) > 2 and img.shape[2] == 4:
        #convert the image from RGBA2RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    xcmin=xc-3
    xcmax=xc+3
    ycmin=yc-3
    ycmax=yc+3
    for x in range(xcmin,xcmax):
        img[ycmin,x]=[0,0,255]
    for x in range(xcmin,xcmax):
        img[ycmax,x]=[0,0,255]

    for y in range(ycmin,ycmax):
        img[y,xcmin]=[0,0,255]
    for y in range(ycmin,ycmax):
        img[y,xcmax]=[0,0,255]
    return img
####################################################
def fonction_p(img, lower_test, upper_test,inverse,w,wh, Nb_sup_wh, mode,List_info_paquets,diam_imp):
    
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)  ##  *** Attention, en Jupyter Ipywidget camera cré un alpha chanel

    modif=adjust_brightness(img, gamma)
    img=modif
    hauteur, largeur = img.shape[:2]
    # sortie d'image avec contours
    image_contours = img.copy()
    # Détection des colonies 
    colors = ['colonies']
    for color in colors:
        # copie de l'image originale
        image_to_process = img.copy()
        # initialisation du compteur
        counter[color] = 0
        # Dépôt des marges (GBR) de détection des couleurs dans un NumPy arrays
        if color == 'colonies':
            lower = np.array(lower_test)
            upper = np.array(upper_test)
        if inverse==1:
            image_to_process = (255-image_to_process)
        else:
            ok=2
        # Recherche des couleurs spécifiées dans les marges dans un masque
        image_mask2 = cv2.cvtColor(image_to_process, cv2.COLOR_BGR2HSV)

        image_mask = cv2.inRange(image_to_process, lower, upper)
        # application du masque
        image_res = cv2.bitwise_or(image_to_process, image_to_process, mask = image_mask)
        ## Transformation de l'image en nuance de gris et flouttage Gaussien
        image_gray = cv2.cvtColor(image_res, cv2.COLOR_BGR2GRAY)
        image_gray = cv2.GaussianBlur(image_gray, (5, 5), 0)
        # Détection des contours, et résolution des imperfections
        image_edged = cv2.Canny(image_gray, 50, 100)
        image_edged = cv2.dilate(image_edged, None, iterations=1)
        image_edged = cv2.erode(image_edged, None, iterations=1)
        
        if mode=="isole":
            cnts, hierarchy = cv2.findContours(image_edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cv2.findContours(image_edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            
        else:
            cnts, hierarchy = cv2.findContours(image_edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            cnts = cv2.findContours(image_edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            #cnts = cv2.findContours(image_edged.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
            ###cnts = cv2.findContours(image_edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
            
            
            #cnts_tot = cv2.findContours(image_edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE) ##
            #cnts_tour = cv2.findContours(image_edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            #cnts = cnts_tot - cnts_tour


            


        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    #####################################
        if mode=="tapis":
            if hierarchy is None:
                ko=2
            else:
                hierar = list(chain.from_iterable(hierarchy))
                lenhierar=len(hierar)
                listi=[]
                for xc in range(0, lenhierar):
                    xcv=hierar[xc]
                    xcvb=hierar[xc][3]
                    if xcvb==-1:
                        listi.append(xc)
    ####
                listi=sorted(listi, reverse=True)
                #print(listi)
                leny=len(listi)
                for a in range(0,leny):
                    u=listi[a]
                    del hierar[u]
                    del cnts[u]
    #####################################
        Aires=[]
        compte=1
        Aire_col_imp=int(10*(pow(diam_imp,2.0627)))  ##float
        Nb_sup_wh=0
        for c in cnts:
        
            # Si le contour n'est pas suffisamment large, l'ignorer : C'est un paramètre à modifier pour les tailles de colonies
            ###if Aire_col_imp<1:
                ###yes=2
            ###else:
            if cv2.contourArea(c) < w or cv2.contourArea(c)>wh:   ##1  #5
                if cv2.contourArea(c)>wh:
                    aire_compt=cv2.contourArea(c)
                    infos=(compte,aire_compt)
                    compte+=1
                    List_info_paquets.append(infos)
                    Nb_sup_wh=Nb_sup_wh+1
                    #print("Nb_sup_wh ligne 117=",Nb_sup_wh)
                continue

            if cv2.contourArea(c)>Aire_col_imp:
                aire_compt=cv2.contourArea(c)
                infos=(compte,aire_compt)
                compte+=1
                List_info_paquets.append(infos)
                Nb_sup_wh=Nb_sup_wh+1
                #print("Nb_sup_wh ligne 126=",Nb_sup_wh)
                #print("List_info_paquets ligne 127=",List_info_paquets)
##############
            Aire=cv2.contourArea(c)
            Aires.append(Aire)
            # Création de la coqie convexe du contour
            hull = cv2.convexHull(c)
            if color == 'colonies':
                # Marquer le contour en vert
                cv2.drawContours(image_contours,[hull],0,coul,1) #0,0,255 # 204,102,0
            counter[color] += 1
            # Décocher le commentaire ci-dessous pour écrire, sur l'image, les TAILLES de colonies détectées
            if affiche_Aire==1:
                cv2.putText(image_contours, "{:.0f}".format(cv2.contourArea(c)), (int(hull[0][0][0]), int(hull[0][0][1])), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 0), 2)
            else:
                ko=1
        Len_Aires=len(Aires)
        if Len_Aires>0:
            Aire_Moy= int(statistics.mean(Aires) )
        else:
            Aire_Moy=0
        Aires=[]
    # Décocher le commentaire ci-dessous pour l'enregistrement de l'image finalisée
    #cv2.imwrite("Capture_denombrement.jpg",image_contours)     ######
    image_contours = image_contours.astype (np.uint8)
    return image_contours , counter[color],Nb_sup_wh,Aire_Moy,List_info_paquets
#####################################################    
def fonction_m(img,x,y, inverse): 
    p_unmin=255
    p_deuxmin=255
    p_troismin=255
    p_unmax=0
    p_deuxmax=0
    p_troismax=0
    if inverse==1:
        img = (255-img)
    else:
        ok=2
    image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)      
    xpmin=x
    xpmax=x+6
    ypmin=y
    ypmax=y+6
    cible_img=image[xpmin:xpmax,ypmin:ypmax]
    for xp in range(xpmin,xpmax-1):
        for yp in range(ypmin,ypmax-1):
            p=img[xp,yp]
            Prep_pixel = np.uint8([[p]])
            hsv_pixel = cv2.cvtColor(Prep_pixel,cv2.COLOR_BGR2HSV)
            pi=hsv_pixel[0]
            p_un=pi[0][0]
            p_deux=pi[0][1]
            p_trois=pi[0][2]
            if p_un<p_unmin:
                p_unmin=p_un
            else:
                z=1
            if p_un>p_unmax:
                p_unmax=p_un
            else:
                z=1
            if p_deux<p_deuxmin:
                p_deuxmin=p_deux
            else:
                z=1
            if p_deux>p_deuxmax:
                p_deuxmax=p_deux
            else:
                z=1 
            if p_trois<p_troismin:
                p_troismin=p_trois
            else:
                z=1
            if p_trois>p_troismax:
                p_troismax=p_trois
            else:
                z=1 
    #lower_test=[p_unmin,25,25]
    #lower_test=[p_unmin,p_deuxmin,p_troismin] ## cool
    lower_test=[p_unmin-32,p_deuxmin,p_troismin]  ###
    #upper_test=[p_unmax,255,255]
    #upper_test=[p_unmax,p_deuxmax,p_troismax]  ## cool
    upper_test=[p_unmax+32,p_deuxmax,p_troismax]  ###
    #print("lower=",lower_test)
    #print("upper=",upper_test)
    return lower_test, upper_test
####################################################
############
def f(change):
    global Nb_sup_wh, List_info_paquets, x2, y2,a, cl , gamma
    
#     a=change['new']
    
    Travail_img(x2,y2,a,cl,gamma,inverse)
    
def Travail_img(x2,y2,a,cl,gamma,inverse): 
    global Nb_sup_wh, List_info_paquets  #  , x2, y2,a
    im_in = PIL.Image.open(io.BytesIO(image_headline2.value))
    im_array = np.array(im_in)#[..., :3]
#     ####
    mask_circle = np.zeros(im_array.shape[:2],np.uint8)
    cv2.circle(mask_circle,(320,240), a.value , 255, -1)
    new = cv2.bitwise_and(im_array, im_array, mask=mask_circle)   
#########################################"
#########################################
    img=new
##############
    img=cible_petite(img,x2,y2)
    lower_test,upper_test=fonction_m(img,x2, y2,inverse)
    img,comptage,Nb_sup_wh,Aire_Moy,List_info_paquets=fonction_p(img, lower_test, upper_test,inverse,w,wh,Nb_sup_wh, mode,List_info_paquets,diam_imp)
###################
###################
    Texte1="On compte : "
    Texte2=" colonies."
    texte2b=" colonie."
    if comptage<2:
        Texte=str(Texte1)+str(comptage)+str(texte2b)
    else:
        Texte=str(Texte1)+str(comptage)+str(Texte2)
    cv2.putText(img, Texte ,(5,350), font, 0.6, (9,183,255), 2); # 0,0,255
##################
    new=img
#########################################
#########################################"   
    fmt = 'png'
    data_comp = compress_to_bytes(new, fmt)
    image_headline.value=data_comp
    #####
    data_comp2 = compress_to_bytes(im_array, fmt)
    image_headline2.value=data_comp2
############
def g(b):
    im_in = PIL.Image.open(io.BytesIO(image_headline2.value))
    im_array = np.array(im_in)#[..., :3]

    img_rgb = cv2.flip(im_array, 1)

    mask_circle = np.zeros(img_rgb.shape[:2],np.uint8)
    cv2.circle(mask_circle,(320,240), a.value, 255, -1)
    new = cv2.bitwise_and(img_rgb, img_rgb, mask=mask_circle)  
    fmt = 'png'
    data_comp = compress_to_bytes(new, fmt)
    image_headline.value=data_comp
    #####
    data_comp = compress_to_bytes(img_rgb, fmt)
    image_headline2.value=data_comp
            #################################
def Recup_image(_):
    im_in = PIL.Image.open(io.BytesIO(image_recorder.image.value))
    im_array = np.array(im_in)#[..., :3]
    ####
    mask_circle = np.zeros(im_array.shape[:2],np.uint8)
    cv2.circle(mask_circle,(320,240), a.value, 255, -1)
    img_rgb = cv2.bitwise_and(im_array, im_array, mask=mask_circle)    
    ####
    fmt = 'png'
    data_comp = compress_to_bytes(img_rgb, fmt)
    image_headline.value = data_comp
#     print("8888888888888888")
    ####
    data_comp2 = compress_to_bytes(im_array, fmt)
    image_headline2.value = data_comp2
                #################################
def compress_to_bytes(data, fmt):
    """
    Helper function to compress image data via PIL/Pillow.
    """
    buff = io.BytesIO()
    img = PIL.Image.fromarray(data)    
    img.save(buff, format=fmt)
    return buff.getvalue()
##########
def arrete_tout(_):
    Widget.close_all()
def update_coords(event):
    global Nb_sup_wh, List_info_paquets, x2, y2,a, cl , gamma
    x2 = event['dataX']
    y2 = event['dataY']

    Travail_img(x2,y2,a,cl,gamma,inverse)
#################
#########################               
def Contrast(_):
    global gamma
    cl=clair_z.value
    fo=sombre_z.value
    cl_new=cl_new_z.value
    fo_new=fo_new_z.value
    if cl>cl_new:
        if fo==0:
            cl_new+=1
            cl_new_z.value=cl_new #
        else:
            clair_z.value=cl_new
            fo_new=fo_new-1
            fo_new_z.value=fo_new
            sombre_z.value=fo_new

    if fo>fo_new:
        if cl==0:
            fo_new+=1
            fo_new_z.value=fo_new
        else:
            sombre_z.value=fo_new
            cl_new=cl_new-1
            cl_new_z.value=cl_new
            clair_z.value=cl_new

    if fo==0:
        gamma=float(cl/10)+1
    if cl==0:
        gamma=1-float(fo/10)
    Travail_img(x2,y2,a,cl,gamma,inverse)
#########################
##################################
def Fonc_isol(change):
    global x2,y2,a,cl,gamma,inverse
    aq = change['new']
    if aq=="La couleur du milieu":
        inverse=1
    else:
        inverse=0
    Travail_img(x2,y2,a,cl,gamma,inverse)
#####################################

In [3]:
affiche_Aire=0
#########################################
  # dictionnaire de comptage des colonies
counter = {}
coul=1
##################################
####### w,wh,Nb_sup_wh, mode,List_info_paquets,diam_imp ###
Aire_col_imp= 60    # int(10*(pow(diam_imp,2.0627)))
diam_imp=round((Aire_col_imp/10)**(1/2.0627))
gamma=1
zoom=1
x = 306
y = 176
x2=274#240
y2=373#320
p_unmin=255
p_deuxmin=255
p_troismin=255
p_unmax=0
p_deuxmax=0
p_troismax=0
zoom2=0
rayon=150
Comptage_min=400
Comptage_max=0
w=1
wh=300
Nb_List=0
Nb_List_max=10
List_max=[]

Nb_List_min=0
List_min=[]
mode="isole"
Nb_sup_wh=0
List_info_paquets=[]
ListeNbColPlus=[]
cl=0
cl_new=0
fo=0
fo_new=0
weballum=0

#################""
font = cv2.FONT_HERSHEY_SIMPLEX  # FONT_HERSHEY_SIMPLEX  # FONT_HERSHEY_PLAIN
inverse=0
x2=274#240
y2=373#320
#######
clair_z = widgets.IntSlider(min = 0, max = 10,  value = 0 , description='+ clair')
sombre_z = widgets.IntSlider(min = 0, max = 10,  value = 0 , description='+ sombre')
Lumino = Box([VBox([clair_z,sombre_z])])
Lumino.layout.border = 'thick double #00CC99' #00CC99
clair_z.observe(Contrast, names='value')
sombre_z.observe(Contrast, names='value')
###
cl_new_z = widgets.IntSlider(min = 0, max = 10,  value = 0 , description='cl_new')
fo_new_z = widgets.IntSlider(min = 0, max = 10,  value = 0 , description='fo_new')
# display(cl_new_z)
# display(fo_new_z)
# #####################################
Calib_widget=widgets.RadioButtons(
            options=[
                "La couleur d'une colonie",
                "La couleur du milieu"
            ],
            layout={'width': 'max-content'}
        )
isol_z=widgets.Box(
    [
        widgets.Label(value='Calibration sur..'),
        Calib_widget
    ]
)
isol_z.layout.border = 'thick double #32a1ce'#'3px solid black'
Calib_widget.observe(Fonc_isol, names='value')
# #####################################
titre = cv2.imread("Titre.jpg", cv2.IMREAD_COLOR)
titre_rgb = cv2.cvtColor(titre, cv2.COLOR_BGR2RGB)
fmt = 'png'
titre_data_comp = compress_to_bytes(titre_rgb, fmt)
titre_w = widgets.Image(value=titre_data_comp,format='jpg',width='640')
# #####################################
img = cv2.imread("Bienvenue.png", cv2.IMREAD_COLOR)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
fmt = 'png'
data_comp = compress_to_bytes(img_rgb, fmt)
image_headline = widgets.Image(value=data_comp,format='jpg',width='640')
image_headline2 = widgets.Image(value=data_comp,format='jpg',width='640')
            #################################
im_events = Event()
im_events.source = image_headline
im_events.watched_events = ['click']
no_drag = Event(source=image_headline, watched_events=['dragstart'], prevent_default_action = True)
im_events.on_dom_event(update_coords)  # im_events.on_dom_event(update_coords)       
            #################################
Texte_w = Label('Bienvenue à tous dans Iris_V3 : La version en ligne !')
Texte_w.layout.border = '3px solid blue'


#             #################################        
a = widgets.IntSlider(min = 150, max = 238,  value = 150 , description='a')
b = widgets.IntSlider(min = 0, max = 1,  value = 0 , description='b')

a.observe(f, names='value')
b.observe(g, names='value')
            #################################

Fenetre=HBox([    VBox([titre_w,image_headline,Texte_w]), VBox([isol_z,a,b,Lumino])    ])
display(Fenetre)
#             #################################   
btn_arret_cam = Button(description="Arreter l'appli",tooltip='Arreter Camera',button_style='info')
btn_arret_cam.on_click(arrete_tout)
display(btn_arret_cam)
camera = CameraStream(constraints={'facing_mode': 'user','audio': False,'video': { 'width': 640, 'height': 480}})
image_recorder = ImageRecorder(stream=camera, tooltip='Cliquer pour tenter une Capture')
btn_stop = Button(description="Ok",tooltip='Cette capture est Ok ?',button_style='danger')
btn_stop.on_click(Recup_image)

yo=HBox([camera,image_recorder, btn_stop])
display(yo)


Button(button_style='info', description="Arreter l'appli", style=ButtonStyle(), tooltip='Arreter Camera')

In [4]:
# Widget.close_all()

In [5]:
# print(counter)

In [6]:
# from ipywidgets import *
# l = Label(' Click or type on me!')
# l.layout.border = '2px solid blue'
# display(l)

In [7]:
# l.value="coucou"